In [1]:
from typing import Tuple, List, Dict, Any, Sequence
import polars as pl
import polars.selectors as cs
import pandas as pd
from pprint import pprint, pformat
import sys
from pathlib import Path
from tqdm import tqdm, trange
import psutil
import re


# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from src.utils import process_lazyframe_in_chunks
from src.loading import DataLoader
from src.preprocess import TextPreprocessor, create_udi_preprocessor, create_company_preprocessor, create_generic_preprocessor
from src.preprocess.preprocess import get_pattern_cols, \
    get_unique_by_cols_safe, get_unique, \
    analyze_null_values, replace_pattern_with_null, overview_col

In [2]:
# maude 데이터 불러오기
loader1 = DataLoader(
    start=2020,
    end=2025,
    output_file = DATA_DIR / 'silver' / 'clean_step3.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader1.load(adapter=adapter, **polars_kwargs)
maude_lf


📖 /Users/eric/yeeun/Project4/data/silver/clean_step3.parquet 로딩 중... (adapter=polars)


In [26]:
MDR_TEXT_PATTERNS = [
    r"^mdr_text_.*_text$"
]

MDR_COLS = get_pattern_cols(maude_lf, MDR_TEXT_PATTERNS)

In [35]:
cols = ['mdr_text_0_text', 'mdr_text_1_text']

mdr_lf = maude_lf.select(pl.col(MDR_COLS))

mdr_concat_lf = mdr_lf.with_columns(
    pl.concat_str(MDR_COLS, separator=' ', ignore_nulls=True).alias('mdr_text')
)

mdr_concat_lf.select('mdr_text').head().collect()

mdr_text
str
"""NO PRODUCT WAS RETURNED FOR EV…"
"""DEPUY SYNTHES IS SUBMITTING TH…"
"""IT WAS REPORTED THAT A MISSING…"
"""THE DEVICE IS EXPECTED TO BE R…"
"""THE PRODUCT WAS NOT RETURNED F…"


In [42]:
# 회사 이름 정제
preprocessor1 = create_generic_preprocessor()
output_path = DATA_DIR / 'silver' / 'mdr_clean.parquet'

preprocessor1.apply_to_lazyframe(
    mdr_concat_lf,
    ['mdr_text'],
    output_path,
    10_000,
)

# del preprocessor1, primary_udi_lf
# output_path.unlink(missing_ok=True)

[GenericText] Extracting unique values from 1 column(s)...
[GenericText] Creating mapping for 1,598,695 unique values...
  - Kept: 1,598,690 (100.0%)
  - Deleted: 5 (0.0%)
Processing 2,627,150 rows in chunks of 10,000...


[GenericText] Processing 1 column(s): 100%|██████████| 263/263 [00:13<00:00, 19.56it/s]


Merging chunks...
✓ Saved to /Users/eric/yeeun/Project4/data/silver/mdr_clean.parquet


In [32]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=output_path,
)

cleaned_mdr_lf = udi_loader.load(adapter, **polars_kwargs)


📖 /Users/eric/yeeun/Project4/data/silver/mdr_clean.parquet 로딩 중... (adapter=polars)


In [40]:
mdr_concat_lf.select(
    pl.col('mdr_text').len().alias('count'),
    pl.col('mdr_text').n_unique().alias('n_unique'),
    # pl.col('mdr_text_0_text').n_unique().alias('one_unique')
).head(10).collect()

count,n_unique
u32,u32
2627150,1598695


In [6]:
analyze_null_values(cleaned_mdr_lf, ['mdr_text_0_text'])
analyze_null_values(maude_lf, ['mdr_text_0_text'])


=== 결측치 분석 ===
전체 행 수: 2,627,150

mdr_text_0_text                              :      1,515개 (  0.06%)

=== 결측치 분석 ===
전체 행 수: 2,627,150

mdr_text_0_text                              :        218개 (  0.01%)


column,null_count,null_pct
str,u32,f64
"""mdr_text_0_text""",218,0.01
